In [1]:
# TODO
#!{__import__('sys').executable} -m pip install --quiet openstudio

In [2]:
import contextlib
import builtins

@contextlib.contextmanager
def temporary_attr(o, name: str):
    a = builtins.getattr(o, name)
    try: yield
    finally: builtins.setattr(o, name, a)

import os
import sys

@contextlib.contextmanager
def temporary_search_path(*paths):
    with temporary_attr(sys, 'path'):
        setattr(sys, 'path', [os.path.expanduser(p) for p in paths])
        try: yield
        finally: pass

In [3]:
pyenergyplus_path = '~/.local/EnergyPlus-23-1-0'
with temporary_search_path(pyenergyplus_path):
    import pyenergyplus as eplus
    import pyenergyplus.api

In [4]:
class EnergyPlusEMS:
    _ep_api = eplus.api.EnergyPlusAPI()

    def __init__(self):
        pass

    def __enter__(self):
        self._ep_state = self._ep_api.state_manager.new_state()
        return self

    def __exit__(self, *_exc_args):
        if getattr(self, '_ep_state', None) is not None:
            self._ep_api.state_manager.delete_state(self._ep_state)

    def _exec(self, *args):
        return self._ep_api.runtime.run_energyplus(
            self._ep_state,
            command_line_args=args
        )
    
with EnergyPlusEMS() as ep_ems:
    ep_ems._exec('--help')


PythonLinkage: Linked to Python Version: "3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:08:06) [GCC 11.3.0]"
Built on Platform: Ubuntu22.04_x86_64
Usage: energyplus [options] [input-file]
Options:
  -a, --annual                 Force annual simulation
  -c, --convert                Output IDF->epJSON or epJSON->IDF, dependent on
                               input file type
  -D, --design-day             Force design-day-only simulation
  -d, --output-directory ARG   Output directory path (default: current
                               directory)
  -h, --help                   Display help information
  -i, --idd ARG                Input data dictionary path (default: Energy+.idd
                               in executable directory)
  -j, --jobs ARG               Multi-thread with N threads; 1 thread with no
                               arg. (Currently only for G-Function generation)
  -m, --epmacro                Run EPMacro prior to simulation
  -p, --output-prefi